In [2]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2021-10-22 11:42:07 +05:30)


In [26]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr

import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.layers import Dense,Dropout,BatchNormalization,Conv1D,Flatten,MaxPooling1D,LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

time: 19.3 s (started: 2021-10-22 11:57:09 +05:30)


In [27]:
start_date=datetime.datetime(2016, 10, 1)
end_date=datetime.datetime(2021,10,15)

time: 0 ns (started: 2021-10-22 11:57:28 +05:30)


In [28]:
df = pdr.get_data_yahoo('GOOGL', start=start_date, end=end_date)
df.drop("Adj Close",axis=1,inplace=True)
print(df.tail())

                   High          Low         Open        Close   Volume
Date                                                                   
2021-10-11  2814.239990  2777.179932  2785.840088  2778.280029   897700
2021-10-12  2790.139893  2715.199951  2789.600098  2728.979980  1816600
2021-10-13  2760.000000  2730.379883  2748.050049  2751.639893   982700
2021-10-14  2826.750000  2776.500000  2789.479980  2823.020020  1594700
2021-10-15  2834.360107  2815.100098  2832.290039  2827.360107  1512900
time: 4.23 s (started: 2021-10-22 11:57:28 +05:30)


In [29]:
start_year=start_date.year
start_month=start_date.month
end_year=end_date.year
end_month=end_date.month

first_days=[]
# First year
for month in range(start_month,13):
    first_days.append(min(df.loc[str(start_year)+"-"+str(month)].index))
# Other years
for year in range(start_year+1,end_year):
    for month in range(1,13):
        first_days.append(min(df.loc[str(year)+"-"+str(month)].index))
# Last year
for month in range(1,end_month+1):
    first_days.append(min(df.loc[str(end_year)+"-"+str(month)].index))

time: 125 ms (started: 2021-10-22 11:57:32 +05:30)


In [30]:
def monthly_df(df):

    dfm=df.resample("M").mean()
    dfm=dfm[:-1] # As we said, we do not consider the month of end_date
    
    dfm["fd_cm"]=first_days[:-1]
    dfm["fd_nm"]=first_days[1:]
    dfm["fd_cm_open"]=np.array(df.loc[first_days[:-1],"Open"])
    dfm["fd_nm_open"]=np.array(df.loc[first_days[1:],"Open"])
    dfm["quot"]=dfm["fd_nm_open"].divide(dfm["fd_cm_open"])
    
    dfm["mv_avg_12"]= dfm["Open"].rolling(window=12).mean().shift(1)
    dfm["mv_avg_24"]= dfm["Open"].rolling(window=24).mean().shift(1)
    
    dfm=dfm.iloc[24:,:] # we remove the first 24 months, since they do not have the 2-year moving average
    
    return dfm

time: 0 ns (started: 2021-10-22 11:57:35 +05:30)


In [34]:
def monthly_df(df):

    dfm=df.resample("M").mean()
    dfm=dfm[:-1] # As we said, we do not consider the month of end_date
    
    dfm["fd_cm"]=first_days[:-1]
    dfm["fd_nm"]=first_days[1:]
    dfm["fd_cm_open"]=np.array(df.loc[first_days[:-1],"Open"])
    dfm["fd_nm_open"]=np.array(df.loc[first_days[1:],"Open"])
    dfm["quot"]=dfm["fd_nm_open"].divide(dfm["fd_cm_open"])
    
    dfm["mv_avg_12"]= dfm["Open"].rolling(window=12).mean().shift(1)
    dfm["mv_avg_24"]= dfm["Open"].rolling(window=24).mean().shift(1)
    
    dfm=dfm.iloc[24:,:] # we remove the first 24 months, since they do not have the 2-year moving average
    
    return dfm

time: 0 ns (started: 2021-10-22 11:58:24 +05:30)


In [36]:
dfm=monthly_df(df)

print(dfm.head())
print(dfm.tail())

ValueError: Length of values (60) does not match length of index (61)

time: 219 ms (started: 2021-10-22 12:10:03 +05:30)
